<a href="https://colab.research.google.com/github/reachel1/Freshman_learn_AI/blob/main/test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Jul 12 06:47:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import glob
# Importing drive method from colab for accessing google drive
from google.colab import drive
# Mounting drive
# This will require authentication : Follow the steps as guided
drive.mount('/Data')

Mounted at /Data


In [ ]:
!ls /Data/'My Drive'/dataset

 cifar-10-batches-py   cifar-10-python.tar.gz  'LeGO-LOAM Public Dataset'


In [ ]:
!pip install gradio
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

In [ ]:
import matplotlib.pyplot as plt # plt 用于显示图片
import matplotlib.image as mpimg # mpimg 用于读取图片
import numpy as np

#resize功能
from scipy import misc
from PIL import Image

# pytorch
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import StepLR

num_classes = 10
# 超参数设置
num_epochs = 100
batch_size = 32
learning_rate = 0.005

In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# 数据预处理
transform0 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])
transform1 = transforms.Compose([
    #图形尺寸填充 填充至36x36
    transforms.Pad(4),
    #随机水平翻转
    transforms.RandomHorizontalFlip(),
    #随机裁剪 裁剪至32x32
    transforms.RandomCrop(32),
    transforms.ToTensor(),
    ])
transform1 = transforms.Compose([
    # 随机亮度调整
    transforms.ColorJitter(brightness=0.2),
    transforms.ToTensor(),
    ])


In [ ]:
LOAD_CIFAR = True
DOWNLOAD_CIFAR = False
# 从data继承读取数据集的类
from torch.utils.data import Dataset, DataLoader

#训练集数据导入 用两种方法实现数据集扩充
train_data0 = torchvision.datasets.CIFAR10(root='/Data/My Drive/dataset',train=True,
                     download=DOWNLOAD_CIFAR, transform=transform0)
train_data1 = torchvision.datasets.CIFAR10(root='/Data/My Drive/dataset',train=True,
                     download=DOWNLOAD_CIFAR, transform=transform1)
train_data2 = torchvision.datasets.CIFAR10(root='/Data/My Drive/dataset',train=True,
                     download=DOWNLOAD_CIFAR, transform=transform2)
concat_trainset = torch.utils.data.ConcatDataset([train_data0, train_data1, train_data2])

# 测试数据集
test_data = torchvision.datasets.CIFAR10(
    root='/Data/My Drive/dataset',
    train=False,
    transform=transform0
)

In [ ]:
# 训练数据加载器
train_loader = torch.utils.data.DataLoader(dataset=concat_trainset,
                                           batch_size=batch_size,
                                           shuffle=True)
# 测试数据加载器
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
##############################
#网络定义Resnet

#定义残差块
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels, use_1x1conv=False, strides=1):  #需要判断是否需要1×1的卷积
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                    kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                    kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                        kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X                                                 ###############################################可以去掉 不存在残差
        return F.relu(Y)



def resnet_block(input_channels, num_channels, num_residuals, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk



class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.b1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
        self.b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
        self.b3 = nn.Sequential(*resnet_block(64, 128, 2))
        self.b4 = nn.Sequential(*resnet_block(128, 256, 2))
        self.b5 = nn.Sequential(*resnet_block(256, 512, 2))
        self.linear = nn.Linear(512, 10)
        self.Aavgpool = nn.AdaptiveAvgPool2d((1,1))

    def forward(self, x):
        x = self.b1(x)
        x = self.b2(x)
        x = self.b3(x)
        x = self.b4(x)
        x = self.b5(x)
        x = self.Aavgpool(x)
        x = torch.flatten(x,1)
        x = self.linear(x)

        return x


In [ ]:
# 实例化一个模型
model = ResNet()

In [ ]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# 自动调整学习率
scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

In [ ]:
# 设置cuda-gpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
!nvidia-smi

cuda:0
Wed Jul 12 08:46:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    28W /  70W |   1161MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+

In [ ]:
# 开始训练
model = model.cuda()
# 存储损失与精度
loss_history = []
accuracy_history = []
loss_times_history = []

total_step = len(train_loader)
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for i, (images, labels) in enumerate(train_loader):

        images = images.cuda()
        labels = labels.cuda()
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #累计损失
        running_loss += loss.item()
        loss_times_history.append(loss.item())
        # 计算准确率
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        if (i+1) % 1000 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    #scheduler.step()
    #计算各epoch中的平均损失值和准确率
    avg_loss = running_loss / len(train_loader)
    accuracy = 100.0 * correct_train / total_train
    #存储平均损失值和准确率
    loss_history.append(avg_loss)
    accuracy_history.append(accuracy)
    #输出学习率
    print(f"Epoch [{epoch+1}/{num_epochs}], Learning Rate: {scheduler.get_lr()[0]}")

Epoch [1/100], Step [1000/10000], Loss: 2.7288
Epoch [1/100], Step [2000/10000], Loss: 2.6874
Epoch [1/100], Step [3000/10000], Loss: 1.8255
Epoch [1/100], Step [4000/10000], Loss: 1.4385
Epoch [1/100], Step [5000/10000], Loss: 2.1798
Epoch [1/100], Step [6000/10000], Loss: 1.6498
Epoch [1/100], Step [7000/10000], Loss: 1.7780
Epoch [1/100], Step [8000/10000], Loss: 1.5906
Epoch [1/100], Step [9000/10000], Loss: 1.3219
Epoch [1/100], Step [10000/10000], Loss: 1.3705
Epoch [2/100], Step [1000/10000], Loss: 1.2136
Epoch [2/100], Step [2000/10000], Loss: 1.7443
Epoch [2/100], Step [3000/10000], Loss: 0.9180
Epoch [2/100], Step [4000/10000], Loss: 0.9620
Epoch [2/100], Step [5000/10000], Loss: 1.6429
Epoch [2/100], Step [6000/10000], Loss: 1.0339
Epoch [2/100], Step [7000/10000], Loss: 1.5249
Epoch [2/100], Step [8000/10000], Loss: 0.7684
Epoch [2/100], Step [9000/10000], Loss: 2.3522
Epoch [2/100], Step [10000/10000], Loss: 0.8433
Epoch [3/100], Step [1000/10000], Loss: 1.5524
Epoch [3/10

In [ ]:
# 输出测试集精度
#总精度及各类精度相关参数定义
correct = 0
total = 0
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
with torch.no_grad():
    for images, labels in test_loader:
        images = images.cuda()
        labels = labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        for label, prediction in zip(labels, predicted):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

print('Accuracy of the net on the train iamges is {:.2f} %'.format(accuracy_history[-1]))
print('Accuracy of the net on the test iamges is {:.2f} %'.format(100 * correct / total))
print('\n')
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy of the net on the train iamges is 76.71 %
Accuracy of the net on the test iamges is 76.44 %


Accuracy for class: plane is 84.0 %
Accuracy for class: car   is 88.0 %
Accuracy for class: bird  is 69.6 %
Accuracy for class: cat   is 61.5 %
Accuracy for class: deer  is 60.6 %
Accuracy for class: dog   is 61.2 %
Accuracy for class: frog  is 86.1 %
Accuracy for class: horse is 81.7 %
Accuracy for class: ship  is 85.5 %
Accuracy for class: truck is 86.2 %


In [ ]:
#  保存模型
torch.save(model.state_dict(), 'model.ckpt')

In [ ]:
# 可视化数据查看
import itertools
def print_label(input_dex):
  plt.figure(figsize=(10, 4))
  plt.subplot(1, 2, 1)
  plt.plot(range(num_epochs), loss_history)
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.title('Loss Curve')

  plt.subplot(1, 2, 2)
  plt.plot(range(num_epochs), accuracy_history)
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.title('Accuracy Curve')

  curve_fig = "curve.png"
  plt.savefig(curve_fig)
  curve = Image.open(curve_fig)
  k = int(int(input_dex)/batch_size)
  m = int(int(input_dex)%batch_size)
  data_iter = iter(test_loader)
  images, labels = next(itertools.islice(data_iter, k, k+1))
  image = images[m].numpy()
  image = np.transpose(image, (1,2,0))
  show_img = image
  imagebatch = image.reshape(-1,3,32,32)
  # 转换为torch tensor
  image_tensor = torch.from_numpy(imagebatch)
  image_tensor = image_tensor.cuda()
  # 调用模型进行评估
  model.eval()
  output = model(image_tensor)
  precise, predicted = torch.max(output.data, 1)
  pre = predicted.cpu().numpy()

  return curve,classes[pre[0]],classes[labels[m].numpy()],show_img


demo = gr.Interface(fn=print_label,
                    inputs=gr.Textbox(label="Input 0-9999"),
                    outputs=[gr.outputs.Image(type="pil",label="loss and acc"),
                    gr.Textbox(label="predict label"),
                    gr.Textbox(label="true label"),
                    gr.outputs.Image(type="pil",label="Image")]
                    )
demo.launch()


NameError: ignored